<a href="https://colab.research.google.com/github/abhishekDey11209/Projects/blob/main/Content_based_recommendation_movie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
#importing Libraries
import pandas as pd
import numpy as np
import difflib      #used to find the close match given by the user
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

Data Collection and Pre Processing

In [2]:
#loading the data in a pandas dataframe
movies_df = pd.read_csv('movies.csv')

#viewing all the rows and columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

movies_df.head(1)

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron


In [3]:
#checking the rows and columns of the dataframe
movies_df.shape

(4803, 24)

In [4]:
#checking the name of the coulmns
movies_df.columns

Index(['index', 'budget', 'genres', 'homepage', 'id', 'keywords',
       'original_language', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'cast', 'crew', 'director'],
      dtype='object')

In [5]:
#selecting features that we need
features = ['genres', 'keywords','overview','tagline', 'title', 'cast', 'director']

In [6]:
#handle missing values (null values replacing with empty string) in a DataFrame using pandas
for i in features:
  movies_df[i] = movies_df[i].fillna('')


In [7]:
movies_df.head(1)

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron


In [8]:
movies_df.dtypes

index                     int64
budget                    int64
genres                   object
homepage                 object
id                        int64
keywords                 object
original_language        object
original_title           object
overview                 object
popularity              float64
production_companies     object
production_countries     object
release_date             object
revenue                   int64
runtime                 float64
spoken_languages         object
status                   object
tagline                  object
title                    object
vote_average            float64
vote_count                int64
cast                     object
crew                     object
director                 object
dtype: object

In [9]:
movies = movies_df[['index', 'title', 'genres', 'keywords','overview','tagline', 'cast', 'director']]

In [10]:
movies.head()

,index,title,genres,keywords,overview,tagline,cast,director
0,0,Avatar,Action Adventure Fantasy Science Fiction,culture clash future space war space colony so...,"In the 22nd century, a paraplegic Marine is di...",Enter the World of Pandora.,Sam Worthington Zoe Saldana Sigourney Weaver S...,James Cameron
1,1,Pirates of the Caribbean: At World's End,Adventure Fantasy Action,ocean drug abuse exotic island east india trad...,"Captain Barbossa, long believed to be dead, ha...","At the end of the world, the adventure begins.",Johnny Depp Orlando Bloom Keira Knightley Stel...,Gore Verbinski
2,2,Spectre,Action Adventure Crime,spy based on novel secret agent sequel mi6,A cryptic message from Bond’s past sends him o...,A Plan No One Escapes,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,Sam Mendes
3,3,The Dark Knight Rises,Action Crime Drama Thriller,dc comics crime fighter terrorist secret ident...,Following the death of District Attorney Harve...,The Legend Ends,Christian Bale Michael Caine Gary Oldman Anne ...,Christopher Nolan
4,4,John Carter,Action Adventure Science Fiction,based on novel mars medallion space travel pri...,"John Carter is a war-weary, former military ca...","Lost in our world, found in another.",Taylor Kitsch Lynn Collins Samantha Morton Wil...,Andrew Stanton


In [86]:
#creating a copy of original dataframe movies and removing the duplicates
movies1 = movies.drop_duplicates(subset='title')

# Reset the index of 'movies1' to start from 0
movies1.reset_index(drop=True, inplace=True)
movies1.shape

(4800, 9)

Feature extraction


In [120]:
#concating the features into new a new columns
movies1['combined_features'] = movies1['title']+ ' ' + movies1['genres']+ ' ' + movies1['keywords']+ ' ' + movies1['overview']+ ' ' + movies1['tagline']+ ' ' + movies1['cast']+ ' ' + movies1['director']

<ipython-input-120-faedb239f4b3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies1['combined_features'] = movies1['title']+ ' ' + movies1['genres']+ ' ' + movies1['keywords']+ ' ' + movies1['overview']+ ' ' + movies1['tagline']+ ' ' + movies1['cast']+ ' ' + movies1['director']


In [121]:
movies1.head(1)

,index,title,genres,keywords,overview,tagline,cast,director,combined_features
0,0,Avatar,Action Adventure Fantasy Science Fiction,culture clash future space war space colony so...,"In the 22nd century, a paraplegic Marine is di...",Enter the World of Pandora.,Sam Worthington Zoe Saldana Sigourney Weaver S...,James Cameron,Avatar Action Adventure Fantasy Science Fictio...


In [89]:
##Alternative way to concat the features

#concatinating all the featurs and adding a new column in movies dataframe
# List of columns to concatenate

#columns_to_concat = ['index', 'title', 'genres', 'keywords', 'overview', 'tagline', 'cast', 'director']



# Create a new column 'combined_text' containing the concatenated text


#movies1['combined_text'] = movies1[columns_to_concat].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)


In [90]:
#converting text data to feature vector
vectorization = TfidfVectorizer()

text_vectorization = vectorization.fit_transform(movies1['combined_features'])

In [91]:
print(text_vectorization)

  (0, 4401)	0.13006950089188127
  (0, 14524)	0.08286143596507557
  (0, 23673)	0.1475420451555069
  (0, 18195)	0.12094030212440991
  (0, 15941)	0.16869730247452544
  (0, 26476)	0.11409820141164839
  (0, 30328)	0.1483012523176797
  (0, 25380)	0.15644906965295569
  (0, 24124)	0.16540277777069537
  (0, 31280)	0.1483012523176797
  (0, 30874)	0.17959832648644158
  (0, 24166)	0.10874742558803825
  (0, 19865)	0.030853707110315515
  (0, 30849)	0.07107589659508648
  (0, 9283)	0.1404442707976338
  (0, 5385)	0.16249680103342592
  (0, 1036)	0.1204367686073487
  (0, 1274)	0.05172109829281316
  (0, 22029)	0.1639070297648596
  (0, 1314)	0.03127990564394797
  (0, 20110)	0.1586935513793829
  (0, 10616)	0.1363226646495867
  (0, 3058)	0.09808026385753582
  (0, 28170)	0.14279579080892266
  (0, 2761)	0.09642509164712386
  :	:
  (4799, 19692)	0.05838588523858007
  (4799, 12234)	0.07491861467551222
  (4799, 30394)	0.07402248265574915
  (4799, 9569)	0.06740462119954434
  (4799, 19938)	0.05488079564340216
  (47

In [92]:
#getting the similarty score using cosine_similarity
cos = cosine_similarity(text_vectorization)

In [93]:
print(cos)

[[1.         0.05401348 0.03085241 ... 0.02435463 0.02809774 0.00665853]
 [0.05401348 1.         0.04099888 ... 0.05355352 0.03667816 0.01552801]
 [0.03085241 0.04099888 1.         ... 0.0227007  0.04256264 0.01229254]
 ...
 [0.02435463 0.05355352 0.0227007  ... 1.         0.02988431 0.03920881]
 [0.02809774 0.03667816 0.04256264 ... 0.02988431 1.         0.02880436]
 [0.00665853 0.01552801 0.01229254 ... 0.03920881 0.02880436 1.        ]]


In [167]:
movie_name = input('entre the name of the movie : ')

entre the name of the movie : avatar


In [168]:
#finding the close match given by the user

movie_titles = movies1['title'].tolist()
find_close_match = difflib.get_close_matches(movie_name,movie_titles)

In [169]:
find_close_match

['Avatar']

In [170]:
#only taking the first value from the list
close_match = find_close_match[0]

In [171]:
#find the index of the movie
dummy_value = movies1[movies1['title'] == close_match]
dummby_value_index  = dummy_value['index'].values[0]

In [172]:
dummby_value_index

0

In [173]:
#getting the similarity score of that perticualr movies
cos_score = list(enumerate(cos[dummby_value_index]))

In [174]:
cos_score.sort(key=lambda x: x[1], reverse=True)

In [175]:
#take top 20 values with higest similarity score
top_20_indexes = cos_score[:20]

In [176]:
top_20_indexes

[(0, 1.0000000000000002),
 (3157, 0.15858622605628608),
 (1914, 0.1545415685079751),
 (1531, 0.15331739921975732),
 (2403, 0.14419682613370055),
 (461, 0.1440448425224968),
 (56, 0.14369015694573214),
 (1352, 0.1386195016327179),
 (94, 0.13708002591426774),
 (661, 0.1350947327494308),
 (239, 0.12918347332336708),
 (1354, 0.12860746162434225),
 (2198, 0.1215084066776279),
 (942, 0.12141979903019626),
 (3603, 0.12083120978379666),
 (305, 0.11976491268797985),
 (3728, 0.11976316373347852),
 (1275, 0.11877293788787076),
 (47, 0.11874528356376353),
 (1951, 0.11756722260800687)]

In [177]:
#extracting the indexes from the list
new_top_20_indexes = []
for index, score in top_20_indexes:
    new_top_20_indexes.append(index)

In [178]:
new_top_20_indexes

[0,
 3157,
 1914,
 1531,
 2403,
 461,
 56,
 1352,
 94,
 661,
 239,
 1354,
 2198,
 942,
 3603,
 305,
 3728,
 1275,
 47,
 1951]

In [179]:
#prining top 20 recommended movies

print("Suggested movies: \n")

for i in new_top_20_indexes:
  recommended_movies = movies[movies['index'] == i]
  print(recommended_movies['title'].values[0])

Suggested movies: 

Avatar
Jab Tak Hai Jaan
Lifeforce
Moonraker
Aliens
Lost in Space
Star Trek Beyond
Gattaca
Guardians of the Galaxy
Zathura: A Space Adventure
Gravity
Space Chimps
Lockout
The Book of Life
Lone Wolf McQuade
Treasure Planet
Whale Rider
Sunshine
Star Trek Into Darkness
Space Dogs


Incorporating all the code in a single cell

In [197]:
movie_name = input('entre the name of the movie : ')
movie_titles = movies1['title'].tolist()
#finding the closest values from the list
find_close_match = difflib.get_close_matches(movie_name,movie_titles)

#only taking the first value from the list
close_match = find_close_match[0]

#find the index of the movie
dummy_value = movies1[movies1['title'] == close_match]
dummby_value_index  = dummy_value['index'].values[0]

#getting the similarity score of that perticualr movies
cos_score = list(enumerate(cos[dummby_value_index]))

#sorting the similarity score in descending order
cos_score.sort(key=lambda x: x[1], reverse=True)

#take top 20 values
top_20_indexes = cos_score[:20]

#extracting the indexes from the list
new_top_20_indexes = []
for index, score in top_20_indexes:
    new_top_20_indexes.append(index)

#prining top 20 recommended movies

print("Suggested movies: \n")

for i in new_top_20_indexes:
  recommended_movies = movies[movies['index'] == i]
  title = recommended_movies['title'].values[0]
  genres = recommended_movies['genres'].values[0]

    # Print the information
  print(f"Title: {title}")
  print(f"Genres: {genres}")
  print("\n")

entre the name of the movie : Iron Man
Suggested movies: 

Title: Iron Man
Genres: Action Science Fiction Adventure


Title: Iron Man 2
Genres: Adventure Action Science Fiction


Title: Iron Man 3
Genres: Action Adventure Science Fiction


Title: Avengers: Age of Ultron
Genres: Action Adventure Science Fiction


Title: Captain America: Civil War
Genres: Adventure Action Science Fiction


Title: The Avengers
Genres: Science Fiction Action Adventure


Title: X-Men
Genres: Adventure Action Science Fiction


Title: Ant-Man
Genres: Science Fiction Action Adventure


Title: X-Men: Apocalypse
Genres: Science Fiction


Title: Once Upon a Time in the West
Genres: Western


Title: Guardians of the Galaxy
Genres: Action Science Fiction Adventure


Title: Anatomy
Genres: Horror Thriller


Title: Spider-Man 2
Genres: Action Adventure Fantasy


Title: Mystery Men
Genres: Adventure Fantasy Action Comedy Science Fiction


Title: X-Men: Days of Future Past
Genres: Action Adventure Fantasy Science Ficti